# Dash Server

## Setup

In [45]:
import numpy as np
import pandas as pd

### Logging

In [46]:
from myutils import customlogging
import importlib
importlib.reload(customlogging)
mylogger = customlogging.create_dual_logger(__name__, 'log/dashlog.log', file_reset=True)
mylogger

03-Jul-20 01:25:52 - __main__ - INFO     - Logger __main__ starting...


<Logger __main__ (DEBUG)>

### Libraries

In [47]:
from myutils import generic, betting, timing, guiserver, importer
importer.reload_utils()

20-07-03 01-25-52 - utils lib reloaded


### Historical List

In [49]:
trading = betting.get_api_client()
trading.login()

historical_queue = betting.get_historical(trading, r'data/bfsamplerace')
historical_list = list(historical_queue.queue)
print(f'Data entries: {len(historical_list)}')

Data entries: 20730


### Writing to file

In [ ]:
import jsonpickle
d = jsonpickle.encode(historical_list)

### Ladder processing

In [9]:
importer.reload_utils()
histProcessor = betting.HistoricalProcessor()
f = timing.decorator_timer(histProcessor.process_runners)
runner_ladders = f(historical_list, betting.TICKS)

20-07-01 16-22-49 - utils lib reloaded
Finished process_runners in 104.0522 seconds


In [ ]:
list(runner_ladders.items())[0][1]['ladder']['available_to_back'].shape

In [ ]:
list(runner_ladders['data'])

In [ ]:
list(runner_ladders.items())[0][1]['ladder']['available_to_back'].flatten().shape

In [ ]:
dts = [r[0].publish_time for r in historical_list]

In [ ]:
ids = [k for k in runner_ladders]

In [ ]:
index = pd.MultiIndex.from_product([dts, betting.TICKS])

In [ ]:
cols = pd.MultiIndex.from_product([ids, histProcessor.book_attrs])

In [ ]:
colVals = []
for id, r in runner_ladders.items():
    l = r['ladder']
    for a in histProcessor.book_attrs:
        colVals.append(l[a].flatten())

In [ ]:
np.transpose(np.array([
    r['ladder'][a].flatten()
    for r in runner_ladders.values()
    for a in histProcessor.book_attrs
]))

In [ ]:
[i for i in range(4, 6) for j in range(2)]

In [ ]:
np.array(colVals).transpose().shape

In [ ]:
np.hstack([[1,2],[3,4]])

In [ ]:
pd.DataFrame(data=[[1,2],[3,4]],dtype=object)

## Indexing book against ticks

Could use numpy or regular lists, but should check speed first

In [19]:
from myutils.timing import decorator_timer
import numpy as np

l = range(100000)
npl = np.array(l)

@decorator_timer
def nptestin(x):
    w = np.where(npl==x)[0]
    if w.shape[0]:
        return w[0]
    else:
        print('not found')
        
@decorator_timer
def testin(x):
    if x in l:
        return l.index(x)
    else:
        print('not found')

In [44]:
testin(999999)

not found
Finished testin in 0.0001 seconds


In [43]:
nptestin(999999)

not found
Finished nptestin in 0.0004 seconds


# Dash

In [10]:
import plotly
import plotly.graph_objs as go
import dash_core_components as dcc
import dash
from dash.dependencies import Input, Output
import dash_html_components as html
import dash_table
import plotly.express as px
import pandas as pd
import numpy as np

In [16]:
app = dash.Dash(__name__)

fig = px.scatter(pd.DataFrame({'x': [1,2,3,4], 'y':[10, 40, 35, 70]}))

d0 = historical_list[0][0]

app.layout = html.Div(className='content-container', children=[
    html.H1('Betfair GUI', className='content-header'),
    html.Div(className="info-container", children=[
        html.Div(historical_list[0][0].market_definition.event_name)
    ]),
    dcc.Graph(
        id='example-graph',
        figure=fig
    ),
])


if __name__ == '__main__':
    app.run_server(debug=False)

01-Jul-20 16:30:54 - __main__ - INFO     - Dash is running on http://127.0.0.1:8050/



Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/



01-Jul-20 16:30:54 - __main__ - INFO     -  Warning: This is a development server. Do not use app.run_server


01-Jul-20 16:30:54 - __main__ - INFO     -  in production, use a production WSGI server like gunicorn instead.



 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Jul/2020 16:30:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2020 16:30:57] "GET /assets/GUI.css?m=1593617437.173114 HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2020 16:30:57] "GET /assets/.ipynb_checkpoints/GUI-checkpoint.css?m=1593617141.9396067 HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2020 16:30:57] "GET /assets/styles.css?m=1593617437.1781104 HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2020 16:30:57] "GET /assets/default.css?m=1593617437.1703696 HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2020 16:30:58] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2020 16:30:58] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2020 16:30:58] "GET /_favicon.ico?v=1.13.3 HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2020 16:31:02] "GET /_dash-component-suites/dash_table/bundle.js.map HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2020 16:31:02] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js.map HTTP/1.